# The refrigerators dealer
An appliance dealer sells small refrigerators in the college market. 
This July, 25 units are on hand. For the next 3 months, the dealer can buy from the manufacturer up to 65 refrigerators each month, and can sell to the students up to 100 units each month at the prices in the table below. 
| Month  |Buy   | Sell  |
|---|---|---|
| Aug  | 60  | 90  | 
| Sep  | 65  | 110  |
|  Oct | 68  | 105  |
The dealer has storage facilities for 45 units but must pay a $7/unit/month storage charge for each refrigerator stored. Determine an optimal buying, selling, and storing plan. 
- Provide a mathematical formulation
- Solve the problem using Gurobi

# Solution
Let $i \in \{1, 2, 3\}$ be the month, and $b_i$,$s_i$, $k_i$ the number of refrigerators bought, sold, and stored, respectively in month $i$. The problem can be formulated as follows:
$$\begin{align}\max & 90s_1 + 110s_2 + 105s_3 −60b_1 −65b_2 −68b_3 + 7(k_1 + k_2 + k_3) \\
\text{s.t.}& k_1 = 25 + b_1 −s_1\\
&k_2 = k_1 + b_2 −s_2 \\
&k_3 = k_2 + b_3 −s_3\\ 
&b_1\leq 65 \\
&b_2 \leq65 \\
&b_3 \leq65 \\
&s_1 \leq100 \\
&s_2 \leq100 \\
&s_3 \leq100 \\
&k_1 \leq45\\ 
&k_2 \leq45 \\
&k_3 \leq 45 \\
&b_i, s_i, k_i \geq 0, \forall i \in\{1, 2, 3\}
\end{align}
$$


To solve the problem, first we import the necessary elements.

In [1]:
from gurobipy import GRB, Model

Then we create the data.

In [2]:
months = [8,9,10]
cost = {8:60,9:65,10:68}
price = {8:90,9:110,10:105}
max_buy = 65
max_sell = 100
storage_capacity = 45
storage_cost = 7
current_stock = 25

We create the Model object

In [3]:
m = Model('ref')

Restricted license - for non-production use only - expires 2023-10-25


We add the decision variables

In [4]:
b = m.addVars(months,lb=0,ub=max_buy,vtype=GRB.CONTINUOUS,name="b")
s = m.addVars(months,lb=0,ub=max_sell,vtype=GRB.CONTINUOUS,name = "s")
k = m.addVars(months,lb=0,ub=storage_capacity,vtype=GRB.CONTINUOUS,name="k")

Now the objective function

In [5]:
expr = 0
for i in months:
    expr = expr + price[i] * s[i] - cost[i] * b[i] - storage_cost * k[i]
m.setObjective(expr,sense=GRB.MAXIMIZE)

And the constraints

In [6]:
for i in months:
    if i == 8:
        m.addConstr(k[i]-b[i]+s[i] == current_stock)
    else:
        m.addConstr(k[i]-k[i-1] - b[i]+s[i] == 0)

Finally, we optimize

In [7]:
m.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 128 physical cores, 128 logical processors, using up to 32 threads
Optimize a model with 3 rows, 9 columns and 11 nonzeros
Model fingerprint: 0xabc59fad
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+00, 1e+02]
  Bounds range     [4e+01, 1e+02]
  RHS range        [2e+01, 2e+01]
Presolve removed 1 rows and 4 columns
Presolve time: 0.03s
Presolved: 2 rows, 5 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.1475000e+04   1.687500e+01   0.000000e+00      0s
       4    9.9950000e+03   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.11 seconds (0.00 work units)
Optimal objective  9.995000000e+03


And we print the solution

In [8]:
print('Objective value: %g' % m.objVal)
for v in m.getVars():
    print('%s %g' % (v.varName, v.x))

Objective value: 9995
b[8] 65
b[9] 65
b[10] 65
s[8] 45
s[9] 100
s[10] 75
k[8] 45
k[9] 10
k[10] 0


The optimal solution has a total profit of \$9995 and consists of selling 45 refrigerators in August, 100 in
September and 75 in October, to buy 65 refrigerators in each month, and to store 45 refrigerators in August
and 10 in September.